In [71]:
import tensorflow as tf
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [72]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, InputSpec
import tensorflow_addons as tfa

In [73]:
from tensorflow.keras.layers import BatchNormalization
BatchNormalization._USE_V2_BEHAVIOR = False
from PIL import Image

In [74]:
# RELU6 Layer
class Relu6(Layer):
    ''' ReLU6 Layer.
    
    Performs ReLU6 activation.
    '''
    
    def __init__(self):
        super(Relu6, self).__init__()
        self.relu6 = tf.nn.relu6
        
    @tf.function
    def call(self, inputs):
        return self.relu6(inputs)

In [75]:
# Batch Normalization Layer
class BatchNorm(Layer):
    ''' Batch Normalization Layer.
        
    Performs Batch Normalization.
    '''
    
    
    def __init__(self, scale=True, center=True):
        super(BatchNorm, self).__init__()        
        #self.bn = tf.keras.layers.BatchNormalization(scale=scale, center=center, trainable=True)
        self.bn = BatchNormalization(scale=scale, center=center, trainable=True)

    #@tf.function
    def call(self, inputs, training=True):
        return self.bn(inputs, training=training)

In [76]:
# 2D Convolution
class Convolution2D(Layer):
    '''Performs 2D Convolution without any activation.
    
    Used for 2D convolution including 1x1 convolution blocks.
    '''
    
    
    def __init__(self, filters, kernel_size, strides, padding):
        super(Convolution2D, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                            strides = strides, padding = padding)
        self.bn = BatchNorm()
        
    @tf.function
    def call(self, inputs):
        
        x = self.conv(inputs)
        x = self.bn(x)
        
        return x

In [77]:
# 2D Convolution, RELU6
class Convolution2D_RELU6(Layer):
    '''Performs 2D Convolution with RELU6 activation.
    
    2D Convolution with RELU6 activation.
    Used mainly for residual blocks in Mobilenet V2.
    '''
    
    
    def __init__(self, filters, kernel_size, strides, padding):
        super(Convolution2D_RELU6, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                            strides = strides, padding = padding)
        
        self.bn = BatchNorm()
        self.act = Relu6()
        
    @tf.function
    def call(self, inputs):
        
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        
        return x

In [78]:
# Average Pooling Layer
class AveragePooling(Layer):
    '''Average Pooling Layer.
    
    Used to perform Average pooling operation over the input tensors.
    '''
    
    
    def __init__(self, pool_size):
        super(AveragePooling, self).__init__()
        
        self.avgpool = tf.keras.layers.AveragePooling2D(pool_size=pool_size, padding="SAME")
        
    @tf.function
    def call(self, inputs):
        
        x = self.avgpool(inputs)
         
        return x

In [79]:
class DenseLayer(Layer):
    '''Dense Layer.
    
    Fully Connected Layer.
    '''
    
    
    def __init__(self, units):
        super(DenseLayer, self).__init__()
        
        self.dense = tf.keras.layers.Dense(units=units,
                                           kernel_initializer=tf.random_normal_initializer(stddev=0.01))
        
    @tf.function
    def call(self, inputs):
        
        x = self.dense(inputs)
        
        return x

In [80]:
class FlattenLayer(Layer):
    '''Flatten Layer.
    
    Used to flatten outputs after Convolutions.
    Dense Layer does not automatically manages the flatten.
    '''
    
    
    def __init__(self):
        super(FlattenLayer, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        
    @tf.function
    def call(self, inputs):
        
        x = self.flatten(inputs)
        
        return x

In [81]:
# Depthwise Convolution
class DepthwiseConvolution(Layer):
    ''' Depthwise Convolution Layer.
    
    Performs Depthwise Convolution with Batch Norm
    '''
    
    
    def __init__(self, kernel_size = 3, strides = 1, padding = "SAME"):
        super(DepthwiseConvolution, self).__init__()
        self.dconv = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides,
                                     depth_multiplier=1,
                                     padding=padding)
        self.bn = BatchNorm()
    
    @tf.function
    def call(self, inputs):
        
        x = self.dconv(inputs)
        x = self.bn(x)
        
        return x

In [82]:
# Separable Convolution
class SeparableConvolution(Layer):
    ''' Separable Convolution Layer.
    
    Performs Separable Convolution.
    '''
    
    
    def __init__(self, filters = 32, kernel_size = 3, strides = 1, padding = "SAME", 
                 depth_multiplier = 1):
        super(SeparableConvolution, self).__init__()
        self.sconv = tf.keras.layers.SeparableConv2D(filters,kernel_size, strides=strides,
                                     depth_multiplier=depth_multiplier,
                                     padding=padding)
        self.bn = BatchNorm()
        self.act = Relu6()
    
    @tf.function
    def call(self, inputs):
        
        x = self.sconv(inputs)
        x = self.bn(x)
        x = self.act(x)
        
        return x 

In [83]:
# Group Normalization
class GroupNorm(Layer):
    ''' Group Normalization Layer.
    
    Divides the channels of your inputs into smaller sub groups 
    and normalizes these values based on their mean and variance.
    '''
    
    
    def __init__(self, groups=5, axis=3):
        super(GroupNorm, self).__init__()
        self.gnorm = tfa.layers.GroupNormalization(groups=groups, axis=axis)
    
    @tf.function
    def call(self, inputs):
        return self.gnorm(inputs)




In [84]:
# Layer to perform Residual Addition for Mobilenet V2
class AdditionLayer(Layer):
    ''' Addition Layer.
    
    Adds Output of Expansion block to inputs in case of Stride 1 Blocks.
    '''
    def __init__(self):
        super(AdditionLayer, self).__init__()
        self.add = tf.keras.layers.Add()
    
    @tf.function
    def call(self, input1, input2):
        return self.add([input1, input2])
    

In [85]:
# Global Average Pooling Layer
class GlobalAveragePooling(Layer):
    '''Global Average Pooling Layer.
    
    Used to perform Global Average pooling operation over the input tensors.
    '''
    
    
    def __init__(self):
        super(GlobalAveragePooling, self).__init__()
        
        self.gpool = tf.keras.layers.GlobalAveragePooling2D()
        
    @tf.function
    def call(self, inputs):
        
        x = self.gpool(inputs)
        
        return x




In [86]:
@tf.function
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [87]:
@tf.function
def _split_divisible(num, num_ways, divisible_by=8):
    """Evenly splits num, num_ways so each piece is a multiple of divisible_by."""
    assert num % divisible_by == 0
    assert num / num_ways >= divisible_by
    # Note: want to round down, we adjust each split to match the total.
    base = num // num_ways // divisible_by * divisible_by
    result = []
    accumulated = 0
    for i in range(num_ways):
        r = base
        while accumulated + r < num * (i + 1) / num_ways:
          r += divisible_by
        result.append(r)
        accumulated += r
    assert accumulated == num
    return result

In [88]:
@tf.function
def _fixed_padding(inputs, kernel_size, rate=1):
    """Pads the input along the spatial dimensions independently of input size.

    Pads the input such that if it was used in a convolution with 'VALID' padding,
    the output would have the same dimensions as if the unpadded input was used
    in a convolution with 'SAME' padding.

    Args:
    inputs: A tensor of size [batch, height_in, width_in, channels].
    kernel_size: The kernel to be used in the conv2d or max_pool2d operation.
    rate: An integer, rate for atrous convolution.

    Returns:
    output: A tensor of size [batch, height_out, width_out, channels] with the
      input, either intact (if kernel_size == 1) or padded (if kernel_size > 1).
    """
    kernel_size_effective = [kernel_size[0] + (kernel_size[0] - 1) * (rate - 1),
                           kernel_size[0] + (kernel_size[0] - 1) * (rate - 1)]
    pad_total = [kernel_size_effective[0] - 1, kernel_size_effective[1] - 1]
    pad_beg = [pad_total[0] // 2, pad_total[1] // 2]
    pad_end = [pad_total[0] - pad_beg[0], pad_total[1] - pad_beg[1]]
    padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg[0], pad_end[0]],
                                  [pad_beg[1], pad_end[1]], [0, 0]])
    return padded_inputs

In [89]:
@tf.function
def expand_input_by_factor(n, divisible_by=8):
    return lambda num_inputs, **_: _make_divisible(num_inputs * n, divisible_by)

In [90]:
class ExpandedConvolutionStride1(Layer):
    ''' Expanded Convolution Layer.
        
    Used for Residual blocks of Mobilenet V2 with Stride 1 Blocks.
    Input -> Expansion Block + Input -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=1, padding="SAME", expansion_factor=6):
        super(ExpandedConvolutionStride1, self).__init__()
        
        self.conv1 = Convolution2D_RELU6(input_filters*expansion_factor, (1, 1), 1, padding)
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), 1, padding)
        self.add = AdditionLayer()

    @tf.function
    def call(self, inputs, training = True):
        
        x = self.conv1(inputs)
        x = self.dconv1(x)
        x = self.conv2(x)
        x = self.add(x, inputs)
        
        return x

In [91]:
class ExpandedConvolutionStride2(Layer):
    ''' Expanded Convolution Layer.
        
    Used for Residual blocks of Mobilenet V2 with Stride 2 Blocks.
    Input -> Expansion Block -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=2, padding="SAME", expansion_factor=6):
        super(ExpandedConvolutionStride2, self).__init__()        
        
        self.conv1 = Convolution2D_RELU6(input_filters*expansion_factor, (1, 1), 1, padding)
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), 1, padding)

    @tf.function
    def call(self, inputs, training = True):
        x = self.conv1(inputs)
        x = self.dconv1(x)
        x = self.conv2(x)
        
        return x

In [92]:
class ExpandedConvolution(Layer):
    ''' Expanded Convolution Layer.
        
    Used for Residual blocks of Mobilenet V2 with Stride 1 Blocks.
    Input -> Expansion Block -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=1, padding="SAME", expansion_factor=6):
        super(ExpandedConvolution, self).__init__()        
        
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), block_stride, padding)
        #self.add = AdditionLayer()

    @tf.function
    def call(self, inputs, training = True):
        
        x = self.dconv1(inputs)
        x = self.conv2(x)
        #x = self.add(x, inputs)
        
        return x

In [93]:
class ExpandedConvolutionDiff(Layer):
    ''' Expanded Convolution Layer Diff.
        
    Used for Residual blocks of Mobilenet V2 with Stride 1 blocks with different channels.
    Used for other than first bottleneck layer.
    Input -> Expansion Block -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=1, padding="SAME", expansion_factor=6):
        super(ExpandedConvolutionDiff, self).__init__()        
        
        self.conv1 = Convolution2D_RELU6(input_filters*expansion_factor, (1, 1), 1, padding)
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), block_stride, padding)
        #self.add = AdditionLayer()

    @tf.function
    def call(self, inputs, training = True):
        
        x = self.conv1(inputs)
        x = self.dconv1(x)
        x = self.conv2(x)
        #x = self.add(x, inputs)
        
        return x

In [161]:
conff=""
def create_conf_head_layers(num_classes):
    """ Create layers for classification
    """
    conf_head_layers = [
        [
        
        layers.Conv2D(6 * num_classes, kernel_size=1,
                      padding='same'),
        DepthwiseConvolution(kernel_size=3),            # for 15th block
        ],
        [layers.Conv2D(6 * num_classes, kernel_size=1,
                      padding='same')]  # for 19th block
    ]

    return conf_head_layers


def create_loc_head_layers():
    """ Create layers for regression
    """
    loc_head_layers = [
        [ 
        layers.Conv2D(6 * 4, kernel_size=1,
                      padding='same'),
        DepthwiseConvolution(kernel_size=3),            # for 15th block
        ],
        [layers.Conv2D(6 * 4, kernel_size=1,
                      padding='same')]  # for 19th block
    ]

    return loc_head_layers


class MobilenetV2(Model):
    ''' Mobilenet V2.
        Mobilenet V2 Layer Architecture.
    '''
    
    def __init__(self, num_outputs):
        super(MobilenetV2, self).__init__()
        self.batch_norm = layers.BatchNormalization(
            beta_initializer='glorot_uniform',
            gamma_initializer='glorot_uniform'
        )
        self.batch_norm_1 = layers.BatchNormalization(
            beta_initializer='glorot_uniform',
            gamma_initializer='glorot_uniform'
        )
        self.num_outputs=num_outputs
        # Layer - 1, Convolution 2D, 32 Output Channels, "SAME" padding
        self.conv1 = Convolution2D(32, (3, 3), (2, 2), "SAME")
        
        # Layer - 2, Inverted Residuals and Linear Bottlenecks
        self.exp1 = ExpandedConvolution(input_filters=32, filters=16, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=1) # Output Channels 16, stride = 1
        
        # Layer - 3, Inverted Residuals and Linear Bottlenecks
        self.exp2 = ExpandedConvolutionStride2(input_filters=16, filters=24, kernel = (3, 3), # Input Channels - 16
                                               expansion_factor=6) # Output Channels 24, stride = 2
        
        # Layer - 4, Inverted Residuals and Linear Bottlenecks
        self.exp3 = ExpandedConvolutionStride1(input_filters=24, filters=24, kernel = (3, 3), # Input Channels - 24
                                               expansion_factor=6) # Output Channels 24, stride = 1
        
        # Layer - 5, Inverted Residuals and Linear Bottlenecks
        self.exp4 = ExpandedConvolutionStride2(input_filters=24, filters=32, kernel = (3, 3), # Input Channels - 24
                                               expansion_factor=6) # Output Channels 32, stride = 2
        
        # Layer - 6, Inverted Residuals and Linear Bottlenecks
        self.exp5 = ExpandedConvolutionStride1(input_filters=32, filters=32, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=6) # Output Channels 32, stride = 1
        
        # Layer - 7, Inverted Residuals and Linear Bottlenecks
        self.exp6 = ExpandedConvolutionStride1(input_filters=32, filters=32, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=6) # Output Channels 32, stride = 1
        
        # Layer - 8, Inverted Residuals and Linear Bottlenecks
        self.exp7 = ExpandedConvolutionStride2(input_filters=32, filters=64, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=6) # Output Channels 64, stride = 2
        
        # Layer - 9, Inverted Residuals and Linear Bottlenecks
        self.exp8 = ExpandedConvolutionStride1(input_filters=64, filters=64, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 64, stride = 1
        # Layer - 10, Inverted Residuals and Linear Bottlenecks
        self.exp9 = ExpandedConvolutionStride1(input_filters=64, filters=64, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 64, stride = 1
        
        # Layer - 11, Inverted Residuals and Linear Bottlenecks
        self.exp10 = ExpandedConvolutionStride1(input_filters=64, filters=64, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 48, stride = 1
        
        # Layer - 12, Inverted Residuals and Linear Bottlenecks
        self.exp11 = ExpandedConvolutionDiff(input_filters=64, filters=96, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 96, stride = 1
        
        # Layer - 13, Inverted Residuals and Linear Bottlenecks
        self.exp12 = ExpandedConvolutionStride1(input_filters=96, filters=96, kernel = (3, 3), # Input Channels - 96
                                               expansion_factor=6) # Output Channels 64, stride = 1
        
        # Layer - 14, Inverted Residuals and Linear Bottlenecks
        self.exp13 = ExpandedConvolutionStride1(input_filters=96, filters=96, kernel = (3, 3), # Input Channels - 96
                                               expansion_factor=6) # Output Channels 96, stride = 1
        
        # Layer - 15, Inverted Residuals and Linear Bottlenecks
        self.exp14 = ExpandedConvolutionStride2(input_filters=96, filters=160, kernel = (3, 3), # Input Channels - 96
                                               expansion_factor=6) # Output Channels 160, stride = 2
        
        # Layer - 16, Inverted Residuals and Linear Bottlenecks
        self.exp15 = ExpandedConvolutionStride1(input_filters=160, filters=160, kernel = (3, 3), # Input Channels - 160
                                               expansion_factor=6) # Output Channels 160, stride = 1
        
        # Layer - 17, Inverted Residuals and Linear Bottlenecks
        self.exp16 = ExpandedConvolutionStride1(input_filters=160, filters=160, kernel = (3, 3), # Input Channels - 160
                                               expansion_factor=6) # Output Channels 160, stride = 1
        
        # Layer - 18, Inverted Residuals and Linear Bottlenecks
        self.exp17 = ExpandedConvolutionDiff(input_filters=160, filters=320, kernel = (3, 3), # Input Channels - 160
                                               expansion_factor=6) # Output Channels 320, stride = 1
        
        
        # Layer - 19, Inverted Residuals and Linear Bottlenecks
        self.conv2 = Convolution2D(1280, (1, 1), (1, 1), "SAME")
        self.conf_head_layers = create_conf_head_layers(num_outputs)
        self.loc_head_layers = create_loc_head_layers()
        
    def compute_heads(self, x, idx):
        """ Compute outputs of classification and regression heads
        Args:
            x: the input feature map
            idx: index of the head layer
        Returns:
            conf: output of the idx-th classification head
            loc: output of the idx-th regression head
        """
        global conff
        for layr in self.conf_head_layers[idx]:
            x= layr(x)
        conf=x
        conf = tf.reshape(conf, [conf.shape[0],-1, self.num_outputs])
        for layr in self.loc_head_layers[idx]:
            x=layr(x)
        
        loc = x
        loc = tf.reshape(loc, [loc.shape[0], -1, 4])

        return conf, loc
    
    def call(self, inputs):
        confs=[]
        locs=[]
        # Layer - 1, 2D Conv - Channels (3 -> 32)
        x = self.conv1(inputs)
#         print("Shape 0 check")
#         print(x.shape)
        
        x = self.exp1(x)
#         print("Shape 1 check")
#         print(x.shape)
        x = self.exp2(x)
#         print("Shape 2 check")
#         print(x.shape)
        x = self.exp3(x)
#         print("Shape 3 check")
#         print(x.shape)
        x = self.exp4(x)
#         print("Shape 4 check")
#         print(x.shape)
        x = self.exp5(x)
#         print("Shape 5 check")
#         print(x.shape)
        x = self.exp6(x)
        x = self.exp7(x)
#         print("Shape 7 check")
#         print(x.shape)
        x = self.exp8(x)
#         print("Shape 8 check")
#         print(x.shape)
        x = self.exp9(x)
#         print("Shape 9 check")
#         print(x.shape)
        x = self.exp10(x)
#         print("Shape 10 check")
#         print(x.shape)
        x = self.exp11(x)
        x = self.exp12(x)
        x = self.exp13(x)
        x = self.exp14(x)
        x = self.exp15(x)
#         print("exp_15.shape----->",x.shape)
        conf, loc = self.compute_heads(self.batch_norm_1(x), 0)
        confs.append(conf)
        locs.append(loc)
        x=self.exp16(x)
        x=self.exp17(x)
        x=self.conv2(x)
#         print("conv2.shape--------->",x.shape)
        conf, loc = self.compute_heads(self.batch_norm(x), 1)
        confs.append(conf)
        locs.append(loc)
        confs = tf.concat(confs, axis=1)
        locs = tf.concat(locs, axis=1)
        
        return confs,locs

In [162]:
# Dummy Data to set the inputs
s = (20, 300, 300, 3)
nx = np.random.rand(*s).astype(np.float32)/ 255
print(nx.shape)

(20, 300, 300, 3)


In [163]:
# MobilenetV2 Model Object
num_outputs = 21 # Output Channels
m2 = MobilenetV2(num_outputs)

In [164]:
# Setting input shape for the model
# Setting input shapes manually, as we are not calling model.fit
c,f=m2(nx)
c

<tf.Tensor: shape=(20, 1200, 21), dtype=float32, numpy=
array([[[-1.32068738e-01, -1.76897720e-01,  7.31251314e-02, ...,
         -3.06147039e-01, -3.57274741e-01,  2.42614746e-01],
        [-2.46845588e-01,  5.38341582e-01, -2.43978828e-01, ...,
          2.28534073e-01, -4.13643330e-01, -3.88145834e-01],
        [-1.58372521e-01, -9.73039865e-02, -3.16752076e-01, ...,
         -7.10458085e-02, -1.91992354e-02,  2.02787787e-01],
        ...,
        [-3.20903584e-02, -3.34217250e-02,  5.62954620e-02, ...,
         -8.57427157e-03, -3.31202745e-02, -7.18008503e-02],
        [-2.71447189e-02, -1.30511373e-02, -9.31114256e-02, ...,
          4.09706123e-02,  2.19982583e-02,  3.63222808e-02],
        [ 1.40197864e-02,  9.09917131e-02, -8.86481628e-03, ...,
          3.83191258e-02,  3.83886099e-02, -4.45495471e-02]],

       [[ 5.91140427e-02,  9.75561216e-02, -3.17894891e-02, ...,
         -3.99547577e-01, -8.99053067e-02,  5.19659519e-02],
        [-2.62768809e-02,  8.65597054e-02, -1.3

In [165]:
def create_ssd(num_classes,
               checkpoint_dir=None,
               checkpoint_path=None):
    """ Create SSD model and load pretrained weights
    Args:
        num_classes: number of classes
        pretrained_type: type of pretrained weights, can be either 'VGG16' or 'ssd'
        weight_path: path to pretrained weights
    Returns:
        net: the SSD model
    """
    net = MobilenetV2(num_outputs)
    return net

In [166]:
ss=create_ssd(21)
locs,confs=ss(nx)
locs.shape,confs.shape



(TensorShape([20, 1200, 21]), TensorShape([20, 1200, 4]))

In [167]:
import itertools
import math
import tensorflow as tf


def generate_default_boxes(config):
    """ Generate default boxes for all feature maps
    Args:
        config: information of feature maps
            scales: boxes' size relative to image's size
            fm_sizes: sizes of feature maps
            ratios: box ratios used in each feature maps
    Returns:
        default_boxes: tensor of shape (num_default, 4)
                       with format (cx, cy, w, h)
    """
    default_boxes = []
    scales = config['SSD']['scales']
    fm_sizes = config['SSD']['fm_sizes']
    ratios = config['SSD']['ratios']
    
    for m, fm_size in enumerate(fm_sizes):
        
        for i, j in itertools.product(range(fm_size), repeat=2):
            k=0
#             print(i,j,fm_size)
            cx = (j + 0.5) / fm_size
            cy = (i + 0.5) / fm_size
            default_boxes.append([
                cx,
                cy,
                math.sqrt(scales[0] * scales[1]),
                math.sqrt(scales[0] * scales[1])
                ])
            k+=1
            for ratio in ratios[m]:
                r = math.sqrt(ratio)
                default_boxes.append([
                    cx,
                    cy,
                    scales[m] * r,
                    scales[m] / r
                ])
                k+=1
#             print(k)

    default_boxes = tf.constant(default_boxes)
    default_boxes = tf.clip_by_value(default_boxes, 0.0, 1.0)
#     print("default_boxes---------------------->",default_boxes.shape)
    return default_boxes

In [168]:
generate_default_boxes(config)

<tf.Tensor: shape=(1200, 4), dtype=float32, numpy=
array([[0.05      , 0.05      , 0.4358899 , 0.4358899 ],
       [0.05      , 0.05      , 0.2       , 0.2       ],
       [0.05      , 0.05      , 0.28284273, 0.14142136],
       ...,
       [0.95      , 0.95      , 0.67175144, 1.        ],
       [0.95      , 0.95      , 1.        , 0.5484828 ],
       [0.95      , 0.95      , 0.5482085 , 1.        ]], dtype=float32)>

In [101]:
import tensorflow as tf


def compute_area(top_left, bot_right):
    """ Compute area given top_left and bottom_right coordinates
    Args:
        top_left: tensor (num_boxes, 2)
        bot_right: tensor (num_boxes, 2)
    Returns:
        area: tensor (num_boxes,)
    """
    # top_left: N x 2
    # bot_right: N x 2
    hw = tf.clip_by_value(bot_right - top_left, 0.0, 512.0)
    area = hw[..., 0] * hw[..., 1]

    return area


def compute_iou(boxes_a, boxes_b):
    """ Compute overlap between boxes_a and boxes_b
    Args:
        boxes_a: tensor (num_boxes_a, 4)
        boxes_b: tensor (num_boxes_b, 4)
    Returns:
        overlap: tensor (num_boxes_a, num_boxes_b)
    """
    # boxes_a => num_boxes_a, 1, 4
#     print("box_a",boxes_a.shape,"box_b",boxes_b.shape)
    boxes_a = tf.expand_dims(boxes_a, 1)

    # boxes_b => 1, num_boxes_b, 4
#     print("transformed_a--------->",boxes_a.shape)
    boxes_b = tf.expand_dims(boxes_b, 0)
#     print("transformed_b--------->",boxes_b.shape)
#     print("boxes_a[..., :2]-------->",boxes_a[..., :].shape)
#     print("boxes_b[..., :2]-------->",boxes_b[..., :].shape)
    top_left = tf.math.maximum(boxes_a[..., :2], boxes_b[..., :2])
    bot_right = tf.math.minimum(boxes_a[..., 2:], boxes_b[..., 2:])
#     print("top_left------------>",top_left.shape,"bot_right------>",bot_right.shape)
    overlap_area = compute_area(top_left, bot_right)
    area_a = compute_area(boxes_a[..., :2], boxes_a[..., 2:])
    area_b = compute_area(boxes_b[..., :2], boxes_b[..., 2:])
#     print("area_a.shape------->",area_a.shape,"area_b.shape--------->",area_b.shape,"overlap_area.shape------->",overlap_area.shape)
    overlap = overlap_area / (area_a + area_b - overlap_area)

    return overlap


def compute_target(default_boxes, gt_boxes, gt_labels, iou_threshold=0.5):
    """ Compute regression and classification targets
    Args:
        default_boxes: tensor (num_default, 4)
                       of format (cx, cy, w, h)
        gt_boxes: tensor (num_gt, 4)
                  of format (xmin, ymin, xmax, ymax)
        gt_labels: tensor (num_gt,)
    Returns:
        gt_confs: classification targets, tensor (num_default,)
        gt_locs: regression targets, tensor (num_default, 4)
    """
    # Convert default boxes to format (xmin, ymin, xmax, ymax)
    # in order to compute overlap with gt boxes
    transformed_default_boxes = transform_center_to_corner(default_boxes)
    iou = compute_iou(transformed_default_boxes, gt_boxes)
#     print("iou--------------->",iou.shape)
    best_gt_iou = tf.math.reduce_max(iou, 1)
#     print("best_gt_iou----------->",best_gt_iou.shape) 
    best_gt_idx = tf.math.argmax(iou, 1)
#     print("best_gt_idx----------->",best_gt_idx.shape) #for every anchor best overlap from all the gt
    best_default_iou = tf.math.reduce_max(iou, 0)
#     print("best_default_iou----------->",best_default_iou.shape)
    best_default_idx = tf.math.argmax(iou, 0)
#     print("best_default_idx----------->",best_default_idx.shape)  #box of iou for every gt for every anchor
#     print(best_default_idx[0],best_gt_idx[best_default_idx[0]])
#     best_gt_idx = tf.tensor_scatter_nd_update(
#         best_gt_idx,
#         tf.expand_dims(best_default_idx, 1),
#         tf.range(best_default_idx.shape[0], dtype=tf.int64))
#     # Normal way: use a for loop
#     # for gt_idx, default_idx in enumerate(best_default_idx):
#     #     best_gt_idx = tf.tensor_scatter_nd_update(
#     #         best_gt_idx,
#     #         tf.expand_dims([default_idx], 1),
#     #         [gt_idx])

#     best_gt_iou = tf.tensor_scatter_nd_update(
#         best_gt_iou,
#         tf.expand_dims(best_default_idx, 1),
#         tf.ones_like(best_default_idx, dtype=tf.float32))

#     print("best_gt_iou-------.....................---->",best_gt_iou.shape)
#     print("gt_labels-----------_>",gt_labels)
    gt_confs = tf.gather(gt_labels, best_gt_idx)   # gt_class contained in each anchor box
#     print("gt_confs-----------_>",gt_confs.shape)
    gt_confs = tf.where(
        tf.less(best_gt_iou, iou_threshold),
        tf.zeros_like(gt_confs),
        gt_confs)

    gt_boxes = tf.gather(gt_boxes, best_gt_idx)     #gt_boxes (xmin,xmax,ymin,ymax) for each achor 
    gt_locs = encode(default_boxes, gt_boxes)

    return gt_confs, gt_locs


def encode(default_boxes, boxes, variance=[0.1, 0.2]):
    """ Compute regression values
    Args:
        default_boxes: tensor (num_default, 4)
                       of format (cx, cy, w, h)
        boxes: tensor (num_default, 4)
               of format (xmin, ymin, xmax, ymax)
        variance: variance for center point and size
    Returns:
        locs: regression values, tensor (num_default, 4)
    """
    # Convert boxes to (cx, cy, w, h) format
    transformed_boxes = transform_corner_to_center(boxes)

    locs = tf.concat([
        (transformed_boxes[..., :2] - default_boxes[:, :2]
         ) / (default_boxes[:, 2:] * variance[0]),
        tf.math.log(transformed_boxes[..., 2:] / default_boxes[:, 2:]) / variance[1]],
        axis=-1)

    return locs


def decode(default_boxes, locs, variance=[0.1, 0.2]):
    """ Decode regression values back to coordinates
    Args:
        default_boxes: tensor (num_default, 4)
                       of format (cx, cy, w, h)
        locs: tensor (batch_size, num_default, 4)
              of format (cx, cy, w, h)
        variance: variance for center point and size
    Returns:
        boxes: tensor (num_default, 4)
               of format (xmin, ymin, xmax, ymax)
    """
    locs = tf.concat([
        locs[..., :2] * variance[0] *
        default_boxes[:, 2:] + default_boxes[:, :2],
        tf.math.exp(locs[..., 2:] * variance[1]) * default_boxes[:, 2:]], axis=-1)

    boxes = transform_center_to_corner(locs)

    return boxes


def transform_corner_to_center(boxes):
    """ Transform boxes of format (xmin, ymin, xmax, ymax)
        to format (cx, cy, w, h)
    Args:
        boxes: tensor (num_boxes, 4)
               of format (xmin, ymin, xmax, ymax)
    Returns:
        boxes: tensor (num_boxes, 4)
               of format (cx, cy, w, h)
    """
    center_box = tf.concat([
        (boxes[..., :2] + boxes[..., 2:]) / 2,
        boxes[..., 2:] - boxes[..., :2]], axis=-1)

    return center_box


def transform_center_to_corner(boxes):
    """ Transform boxes of format (cx, cy, w, h)
        to format (xmin, ymin, xmax, ymax)
    Args:
        boxes: tensor (num_boxes, 4)
               of format (cx, cy, w, h)
    Returns:
        boxes: tensor (num_boxes, 4)
               of format (xmin, ymin, xmax, ymax)
    """
    corner_box = tf.concat([
        boxes[..., :2] - boxes[..., 2:] / 2,
        boxes[..., :2] + boxes[..., 2:] / 2], axis=-1)

    return corner_box





In [169]:
import tensorflow as tf


def hard_negative_mining(loss, gt_confs, neg_ratio):
    """ Hard negative mining algorithm
        to pick up negative examples for back-propagation
        base on classification loss values
    Args:
        loss: list of classification losses of all default boxes (B, num_default)
        gt_confs: classification targets (B, num_default)
        neg_ratio: negative / positive ratio
    Returns:
        pos_idx: positive samples
        neg_idx:negative samples
    """
    # loss: B x N
    # gt_confs: B x N
    pos_idx = gt_confs > 0
#     print("pos_idx----------------->",pos_idx)
#     print("gt_confs.shape----------------->",gt_confs.shape)
    num_pos = tf.reduce_sum(tf.dtypes.cast(pos_idx, tf.int32), axis=1)
    num_neg = num_pos * neg_ratio
#     print("num_neg.shape----------------->",num_neg.shape)
#     print("loss.shape",loss.shape)
    rank = tf.argsort(loss, axis=1, direction='DESCENDING')  #boxes having more loss indices sorted desecnding (box numbers)
#     print("rankk----->",rank,"dsasdasdas")
    rank = tf.argsort(rank, axis=1)                          #indices of boxes present where in the array soreted acctoring to index
#     print("duii------>",rank.numpy())
    neg_idx = rank < tf.expand_dims(num_neg, 1)              
#     print("neg_idx--------->",neg_idx)
    return pos_idx, neg_idx


class SSDLosses(object):
    """ Class for SSD Losses
    Attributes:
        neg_ratio: negative / positive ratio
        num_classes: number of classes
    """

    def __init__(self, neg_ratio, num_classes):
        self.neg_ratio = neg_ratio
        self.num_classes = num_classes

    def __call__(self, confs, locs, gt_confs, gt_locs):
        """ Compute losses for SSD
            regression loss: smooth L1
            classification loss: cross entropy
        Args:
            confs: outputs of classification heads (B, num_default, num_classes)
            locs: outputs of regression heads (B, num_default, 4)
            gt_confs: classification targets (B, num_default)
            gt_locs: regression targets (B, num_default, 4)
        Returns:
            conf_loss: classification loss
            loc_loss: regression loss
        """
        cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='none')

        # compute classification losses
        # without reduction
#         print(confs.shape,gt_confs.shape)
        temp_loss = cross_entropy(
            gt_confs, confs)
        pos_idx, neg_idx = hard_negative_mining(
            temp_loss, gt_confs, self.neg_ratio)

        # classification loss will consist of positive and negative examples

        cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='sum')
        smooth_l1_loss = tf.keras.losses.Huber(reduction='sum')

        conf_loss = cross_entropy(
            gt_confs[tf.math.logical_or(pos_idx, neg_idx)],
            confs[tf.math.logical_or(pos_idx, neg_idx)])

        # regression loss only consist of positive examples
        loc_loss = smooth_l1_loss(
            # tf.boolean_mask(gt_locs, pos_idx),
            # tf.boolean_mask(locs, pos_idx))
            gt_locs[pos_idx],
            locs[pos_idx])

        num_pos = tf.reduce_sum(tf.dtypes.cast(pos_idx, tf.float32))
#         print("loss_function------------>",conf_loss.numpy(),loc_loss.numpy(),temp_loss.numpy())
        conf_loss = conf_loss / num_pos
        loc_loss = loc_loss / num_pos

        return conf_loss, loc_loss


def create_losses(neg_ratio, num_classes):
    criterion = SSDLosses(neg_ratio, num_classes)

    return criterion

In [170]:
from functools import partial


class VOCDataset():
    """ Class for VOC Dataset
    Attributes:
        root_dir: dataset root dir (ex: ./data/VOCdevkit)
        year: dataset's year (2007 or 2012)
        num_examples: number of examples to be used
                      (in case one wants to overfit small data)
    """

    def __init__(self, root_dir, year, default_boxes,
                 new_size, num_examples=-1, augmentation=None):
        super(VOCDataset, self).__init__()
        self.idx_to_name = [
            'aeroplane', 'bicycle', 'bird', 'boat',
            'bottle', 'bus', 'car', 'cat', 'chair',
            'cow', 'diningtable', 'dog', 'horse',
            'motorbike', 'person', 'pottedplant',
            'sheep', 'sofa', 'train', 'tvmonitor']
        self.name_to_idx = dict([(v, k)
                                 for k, v in enumerate(self.idx_to_name)])

        self.data_dir = os.path.join(root_dir, 'VOC{}'.format(year))
        self.image_dir = os.path.join(self.data_dir, 'JPEGImages')
        self.anno_dir = os.path.join(self.data_dir, 'Annotations')
        self.ids = list(map(lambda x: x[:-4], os.listdir(self.image_dir)))
        self.default_boxes = default_boxes
        self.new_size = new_size

        if num_examples != -1:
            self.ids = self.ids[:num_examples]

        self.train_ids = self.ids[:int(len(self.ids) * 0.75)]
        self.val_ids = self.ids[int(len(self.ids) * 0.75):]

        if augmentation == None:
            self.augmentation = ['original']
        else:
            self.augmentation = augmentation + ['original']

    def __len__(self):
        return len(self.ids)

    def _get_image(self, index):
        """ Method to read image from file
            then resize to (300, 300)
            then subtract by ImageNet's mean
            then convert to Tensor
        Args:
            index: the index to get filename from self.ids
        Returns:
            img: tensor of shape (3, 300, 300)
        """
        filename = self.ids[index]
        img_path = os.path.join(self.image_dir, filename + '.jpg')
        img = Image.open(img_path)

        return img

    def _get_annotation(self, index, orig_shape):
        """ Method to read annotation from file
            Boxes are normalized to image size
            Integer labels are increased by 1
        Args:
            index: the index to get filename from self.ids
            orig_shape: image's original shape
        Returns:
            boxes: numpy array of shape (num_gt, 4)
            labels: numpy array of shape (num_gt,)
        """
        h, w = orig_shape
        filename = self.ids[index]
        anno_path = os.path.join(self.anno_dir, filename + '.xml')
        objects = ET.parse(anno_path).findall('object')
        boxes = []
        labels = []

        for obj in objects:
            name = obj.find('name').text.lower().strip()
            bndbox = obj.find('bndbox')
            xmin = (float(bndbox.find('xmin').text) - 1) / w
            ymin = (float(bndbox.find('ymin').text) - 1) / h
            xmax = (float(bndbox.find('xmax').text) - 1) / w
            ymax = (float(bndbox.find('ymax').text) - 1) / h
            boxes.append([xmin, ymin, xmax, ymax])

            labels.append(self.name_to_idx[name] + 1)

        return np.array(boxes, dtype=np.float32), np.array(labels, dtype=np.int64)

    def generate(self, subset=None):
        """ The __getitem__ method
            so that the object can be iterable
        Args:
            index: the index to get filename from self.ids
        Returns:
            img: tensor of shape (300, 300, 3)
            boxes: tensor of shape (num_gt, 4)
            labels: tensor of shape (num_gt,)
        """
        if subset == 'train':
            indices = self.train_ids
        elif subset == 'val':
            indices = self.val_ids
        else:
            indices = self.ids
        for index in range(len(indices)):
            # img, orig_shape = self._get_image(index)
            filename = indices[index]
            img = self._get_image(index)
            w, h = img.size
            boxes, labels = self._get_annotation(index, (h, w))
            boxes = tf.constant(boxes, dtype=tf.float32)
            labels = tf.constant(labels, dtype=tf.int64)
#             print(labels)

            augmentation_method = np.random.choice(self.augmentation)
            if augmentation_method == 'patch':
                img, boxes, labels = random_patching(img, boxes, labels)
            elif augmentation_method == 'flip':
                img, boxes, labels = horizontal_flip(img, boxes, labels)

            img = np.array(img.resize(
                (self.new_size, self.new_size)), dtype=np.float32)
            img = (img / 127.0) - 1.0
            img = tf.constant(img, dtype=tf.float32)

            gt_confs, gt_locs = compute_target(
                self.default_boxes, boxes, labels)

            yield filename, img, gt_confs, gt_locs


def create_batch_generator(root_dir, year, default_boxes,
                           new_size, batch_size, num_batches,
                           mode,
                           augmentation=None):
    num_examples = batch_size * num_batches if num_batches > 0 else -1
    voc = VOCDataset(root_dir, year, default_boxes,
                     new_size, num_examples, augmentation)

    info = {
        'idx_to_name': voc.idx_to_name,
        'name_to_idx': voc.name_to_idx,
        'length': len(voc),
        'image_dir': voc.image_dir,
        'anno_dir': voc.anno_dir
    }

    if mode == 'train':
        train_gen = partial(voc.generate, subset='train')
        train_dataset = tf.data.Dataset.from_generator(
            train_gen, (tf.string, tf.float32, tf.int64, tf.float32))
        val_gen = partial(voc.generate, subset='val')
        val_dataset = tf.data.Dataset.from_generator(
            val_gen, (tf.string, tf.float32, tf.int64, tf.float32))

        train_dataset = train_dataset.shuffle(40).batch(batch_size)
        val_dataset = val_dataset.batch(batch_size)

        return train_dataset.take(num_batches), val_dataset.take(-1), info
    else:
        dataset = tf.data.Dataset.from_generator(
            voc.generate, (tf.string, tf.float32, tf.int64, tf.float32))
        dataset = dataset.batch(batch_size)
        return dataset.take(num_batches), info

In [171]:
import argparse
import tensorflow as tf
import os
import sys
import time
import yaml

from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
config={
    'SSD':{
          'ratios': [[1.0,2.0,0.5,3.0,.333], [1.0,2.0,0.5,3.0,.333]],
          'scales': [0.2, 0.95],
          'fm_sizes': [10, 10],
          'image_size': 300,
            },
    'batch_size':64,
    'data_year':'2007',
    'data_dir':"./",
    'num_batches':-1,
    'neg_ratio':3,
    'initial_lr':1e-3,
    'momentum':0.9,
    'weight_decay':5e-4,
    'num_epochs':120,
    'checkpoint_dir':'checkpoints',
    'pretrained_type':'base',
    
}






In [172]:
import os
NUM_CLASSES = 21

os.makedirs(config['checkpoint_dir'], exist_ok=True)

# @tf.function()
def train_step(imgs, gt_confs, gt_locs, ssd, criterion, optimizer,config):
    with tf.GradientTape() as tape:
        confs, locs = ssd(imgs)
#         print(confs)
#         print("gt_confs.shape------------------>",gt_confs.shape)
#         print("gt_locs.shape------------------->",gt_locs.shape)
#         print("real_confs.shape--------------->",confs.shape)
#         print("real_locs.shape---------------_>",locs.shape)
        conf_loss, loc_loss = criterion(
            confs, locs, gt_confs, gt_locs)
#         print("train_loss---------->",conf_loss,loc_loss,"dasdasdadasdas")
        loss = conf_loss + loc_loss
        l2_loss = [tf.nn.l2_loss(t) for t in ssd.trainable_variables]
        l2_loss = config['weight_decay'] * tf.math.reduce_sum(l2_loss)
        loss += l2_loss
#         print("total_loss.shape------------------>",loss.shape)
#     print(loss.numpy())
    gradients = tape.gradient(loss, ssd.trainable_variables)
    optimizer.apply_gradients(zip(gradients, ssd.trainable_variables))

    return loss, conf_loss, loc_loss, l2_loss





default_boxes = generate_default_boxes(config)


batch_generator, val_generator, info = create_batch_generator(
    config['data_dir'],config['data_year'], default_boxes,
    config['SSD']['image_size'],
    config['batch_size'],config['num_batches'],
    mode='train', augmentation=None)  # the patching algorithm is currently causing bottleneck sometimes
# print("info_length------------------->",info['length'])
try:
    ssd = create_ssd(NUM_CLASSES)
    print("ssd_created")
    print(ssd)
except Exception as e:
    print(e)
    print('The program is exiting...')
    sys.exit()

criterion = create_losses(config['neg_ratio'], NUM_CLASSES)

steps_per_epoch = info['length'] // config['batch_size']

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
lr_fn = PiecewiseConstantDecay(
    boundaries=[int(steps_per_epoch * config['num_epochs'] * 2 / 3),
                int(steps_per_epoch * config['num_epochs'] * 5 / 6)],
    values=[config['initial_lr'],config['initial_lr'] * 0.1, config['initial_lr'] * 0.01])

optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr_fn,
    momentum=config['momentum'])



train_log_dir = 'logs/train'
val_log_dir = 'logs/val'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

for epoch in range(config['num_epochs']):
    avg_loss = 0.0
    avg_conf_loss = 0.0
    avg_loc_loss = 0.0
    start = time.time()
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(batch_generator):
#         print(gt_confs.shape,imgs.shape)
        loss, conf_loss, loc_loss, l2_loss = train_step(
            imgs, gt_confs, gt_locs, ssd, criterion, optimizer,config)
        avg_loss = (avg_loss * i + loss.numpy()) / (i + 1)
        avg_conf_loss = (avg_conf_loss * i + conf_loss.numpy()) / (i + 1)
        avg_loc_loss = (avg_loc_loss * i + loc_loss.numpy()) / (i + 1)
#         print(i)
#         print("train------------------------_________>",avg_loss,avg_conf_loss,avg_loc_loss)
        if (i + 1) % 50 == 0:
            print('Epoch: {} Batch {} Time: {:.2}s | Loss: {:.4f} Conf: {:.4f} Loc: {:.4f}'.format(
                epoch + 1, i + 1, time.time() - start, avg_loss, avg_conf_loss, avg_loc_loss))

    avg_val_loss = 0.0
    avg_val_conf_loss = 0.0
    avg_val_loc_loss = 0.0
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(val_generator):
        val_confs, val_locs = ssd(imgs)
        val_conf_loss, val_loc_loss = criterion(
            val_confs, val_locs, gt_confs, gt_locs)
        val_loss = val_conf_loss + val_loc_loss
        avg_val_loss = (avg_val_loss * i + val_loss.numpy()) / (i + 1)
        avg_val_conf_loss = (avg_val_conf_loss * i + val_conf_loss.numpy()) / (i + 1)
        avg_val_loc_loss = (avg_val_loc_loss * i + val_loc_loss.numpy()) / (i + 1)

    with train_summary_writer.as_default():
        tf.summary.scalar('loss', avg_loss, step=epoch)
        tf.summary.scalar('conf_loss', avg_conf_loss, step=epoch)
        tf.summary.scalar('loc_loss', avg_loc_loss, step=epoch)

    with val_summary_writer.as_default():
        tf.summary.scalar('loss', avg_val_loss, step=epoch)
        tf.summary.scalar('conf_loss', avg_val_conf_loss, step=epoch)
        tf.summary.scalar('loc_loss', avg_val_loc_loss, step=epoch)
    print(epoch)
    if (epoch + 1) % 10 == 0:
        ssd.save_weights(
            os.path.join(config['checkpoint_dir'], 'ssd_epoch_{}.h5'.format(epoch + 1)))

ssd_created
(64, 1200, 21) (64, 1200)
0
(64, 1200, 21) (64, 1200)
1
(64, 1200, 21) (64, 1200)
2
(64, 1200, 21) (64, 1200)
3
(64, 1200, 21) (64, 1200)
4
(64, 1200, 21) (64, 1200)


KeyboardInterrupt: 

In [ ]:
a=[[11,2,3],[3,7,9]]

In [65]:
a_np=np.array(a)
nx_tf=tf.convert_to_tensor(a_np)
tf.math.argmax(a,1)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 2])>

In [70]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [107]:
from tensorflow.keras import backend as K

# Create, compile and train model...

frozen_graph = freeze_session(tf.compat.v1.keras.backend.get_session(),output_names=[out.op.name for out in model.outputs])

AttributeError: module 'tensorflow.keras.backend' has no attribute 'get_session'

In [109]:
import tensorflow.python.keras.backend as K
sess = K.get_session()

AttributeError: module 'tensorflow' has no attribute 'python'